# 빅데이터 분석기사 실기 기출 5회 작업형 2 
[캐글 경진대회 링크](https://www.kaggle.com/competitions/big-data-analytics-certification-kr-2023-5th/)  
[캐글 노트북 공유 링크](https://www.kaggle.com/code/minjunim/5-xgb-rmse-1240) 

## [가격 예측] 중고 자동차
- 자동차 가격을 예측해주세요!
- 예측할 값(y): price
- 평가: RMSE (Root Mean Squared Error)
- data: train.csv, test.csv

제출 형식: result.csv파일을 아래와 같은 형식(수치형)으로 제출
```
pred
11000
20500
19610
...    
11995
```

## 시험 환경처럼 진행

In [46]:
import pandas as pd
import numpy as np

# 0. 데이터 로드
train = pd.read_csv("./Data/train_5.csv")
test = pd.read_csv("./Data/test_5.csv")

print(train.shape)      # (3759, 9)
print(train.head(), "\n")
print(test.shape)
print(test.head(), "\n")



# 1. EDA
# 1-1) train EDA
print(train.info(), "\n")
print(train.isnull().sum(), "\n")     # 결측치 존재 X
print(train.describe(), "\n")

train = train[train["year"] != 2060]     # 중고데이터 이므로 2060년 데이터 삭제
print(train.shape)     # (3758, 9)

# 1-2) test EDA
print(test.info(), "\n")
print(test.isnull().sum(), "\n")      # 결측치 존재 X
print(test.describe(), "\n")

# 1-3) train, test nunique
print(train.describe(include="O"), "\n")     # 이상 없음. train, test nunique 동일  
print(train.describe(include="O"), "\n")



# 2. 전처리
# 2-1) 결측치 처리, id 같은 불필요 칼럼 정리, X와 y 데이터 분리
# 결측치, id같은 불필요 칼럼은 따로 없다고 판단

# X, y 분리
y = train.pop("price")
print(train.head(1), "\n")
print(y.head(1), "\n")

# 2-2) 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

con_cols = train.select_dtypes(exclude="object").copy().columns

train[con_cols] = scaler.fit_transform(train[con_cols])
test[con_cols] = scaler.transform(test[con_cols])

print(train.head(3), "\n")
print(test.head(3), "\n")

# 2-3) 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

print(train.head(3), "\n")
print(test.head(3), "\n")



# 3. 검증 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train, y, test_size=0.2, random_state=2023)
print(X_train.shape, X_val.shape, "\n")



# 4. 모델링 - 회귀
# 4-1) 성능지표 : rmse
from sklearn.metrics import mean_squared_error, r2_score

# 4-2) 랜덤포레스트 회귀
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=2023)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_pred))
print("랜덤포레스트 rmse : ", rf_rmse) # 1285 (2060년 데이터 삭제 전 : 1243)

# 4-3) lgbm 회귀
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(random_state=2023, max_depth=-1, n_estimators=200) 
lgbm.fit(X_train, y_train)
lgbm_pred = lgbm.predict(X_val)
lgbm_rmse = np.sqrt(mean_squared_error(y_val, lgbm_pred))
print("lgbm rmse : ", lgbm_rmse) # 1249 (2060년 데이터 삭제 전 : 1257)

pred = lgbm.predict(test)


# 5. 제출 : df, csv
submit = pd.DataFrame({"pred" : pred})
submit.to_csv("수험번호.csv", index=False)
check = pd.read_csv("수험번호.csv")
print(check.head())

(3759, 9)
       model  year  price transmission  mileage fuelType  tax   mpg  \
0   EcoSport  2017  10298       Manual    25013   Petrol  150  53.3   
1      Focus  2016  10491       Manual    30970   Diesel    0  74.3   
2      S-MAX  2017  13498       Manual    60200   Diesel  150  56.5   
3     Fiesta  2018  10600       Manual    27380   Petrol  145  56.5   
4     Fiesta  2018  11000       Manual     7724   Petrol  145  65.7   

   engineSize  
0         1.0  
1         1.5  
2         2.0  
3         1.0  
4         1.0   

(1617, 8)
     model  year transmission  mileage fuelType  tax   mpg  engineSize
0    S-MAX  2016       Manual    23905   Diesel  125  56.5         2.0
1   Fiesta  2018       Manual    16895   Petrol  145  40.3         1.5
2     Kuga  2017       Manual    12109   Petrol  165  45.6         1.5
3     Kuga  2018    Semi-Auto    13940   Diesel  145  58.9         1.5
4   Fiesta  2013       Manual    31690   Petrol   30  54.3         1.2 

<class 'pandas.core.frame.D

### 참고 : 그리드 서치 결과
#### scoring = "neg_root_mean_squared_error" (rmse)
- 그리드 서치 rmse 활용하려면, neg 붙여줘야함
- rmse는 값이 작을수록 성능이 좋은 것이므로 !

In [47]:
from sklearn.metrics import mean_squared_error, r2_score

# rf
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state=2023)

# lgbm
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor(random_state=2023)


# Hyperparameter tuning

models = [rf, lgbm]

from sklearn.model_selection import GridSearchCV

params = {"n_estimators":[100,200,300], "max_depth":[1,2,3]}

best_models = []

for model in models:
    gs = GridSearchCV(model, param_grid=params, cv = 5, scoring="neg_root_mean_squared_error", n_jobs=4)
    gs.fit(X_train, y_train)
    
    best_model = gs.best_estimator_  # 최적의 모델
    
    # 검증 데이터에 대한 예측 수행
    y_pred = best_model.predict(X_val)

    # RMSE 계산
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    
    print(f"="*10)
    print(f"model : {model}")
    print(f"params : {gs.best_params_}")
    print(f"train RMSE : {-gs.best_score_}")
    print(f"validation RMSE : {rmse}")

model : RandomForestRegressor(random_state=2023)
params : {'max_depth': 3, 'n_estimators': 100}
train RMSE : 2478.044617261835
validation RMSE : 2471.2391742530617
model : LGBMRegressor(random_state=2023)
params : {'max_depth': 3, 'n_estimators': 300}
train RMSE : 1353.6391577953898
validation RMSE : 1315.6999487170838


최적화된 파라미터가 학습 데이터에 과적합(overfitting)되어 일반화 능력 저하 가능성
- 그리드 서치는 학습 데이터를 기반으로 모델의 파라미터 조합을 탐색하여 최적의 조합을 탐색
- 그러나 학습 데이터에 대해서는 최적의 조합일지라도, 검증용 데이터나 새로운 데이터에 대해서는 일반화 성능이 좋지 않을 수 있음
- 모델이 학습 데이터에 지나치게 맞추어져서 새로운 패턴이나 변동성을 제대로 반영하지 못하는 경우가 있기 때문

그리고 RandomForestRegressor() 디폴트 : max_depth=None, n_estimators=100

### 참고 : 그리드 서치 결과, scoring = "r2" (r2_score)
```
====================
model :RandomForestRegressor(random_state=1)
params : {'max_depth': 3, 'n_estimators': 300}
score : 0.7462278481496016
====================
model :LGBMRegressor(random_state=1)
params : {'max_depth': 3, 'n_estimators': 300}
score : 0.9250846831931974
```